In [1]:
import pandas as pd
import numpy as np

### Создание датасета

In [2]:
train_interaction = {  
    'Anton' : {'All along the watchtower' : 1, 'Kalinka-Malinka':1, 'Nothing else matters' : 1, 'Smells Like Teen Spirit' : 1},
    'Lesha' : {'Kalinka-Malinka':1, 'Ty nesi menya reka' :1,'Nothing else matters':1, 'Techet Reka Volga' : 1 },
    'Anna' : {'Nothing else matters' : 1, 'Tam gde klen shumit' :1, 'All along the watchtower' : 1},
    'Dasha' : {'All along the watchtower' : 1, 'Look at me now' : 1, 'Smells Like Teen Spirit' : 1, 'All along the watchtower' : 1},
    'Sasha' : {'Look at me now':1, 'Techet Reka Volga' : 1 , 'Nothing else matters':1,  'Tam gde klen shumit' :1}}


In [3]:
test_interaction = {'query_id': ['Anton', 'Lesha', 'Anna', 'Dasha', 'Sasha'],
 'item_id': ['All along the watchtower', 'Tam gde klen shumit', 'Nothing else matters', 'Nothing else matters', 'Ty nesi menya reka'],
 'ratings' : [1, 1, 1, 1, 1]}

In [4]:
train = pd.DataFrame(columns = ['query_id', 'item_id', 'rating'])
index = 0
for name in train_interaction.keys():
    for value in train_interaction[name]:
        train.loc[index] = [name, value, '1']
        index += 1

train

,query_id,item_id,rating
0,Anton,All along the watchtower,1
1,Anton,Kalinka-Malinka,1
2,Anton,Nothing else matters,1
3,Anton,Smells Like Teen Spirit,1
4,Lesha,Kalinka-Malinka,1
5,Lesha,Ty nesi menya reka,1
6,Lesha,Nothing else matters,1
7,Lesha,Techet Reka Volga,1
8,Anna,Nothing else matters,1
9,Anna,Tam gde klen shumit,1


In [5]:
test = pd.DataFrame(test_interaction)
test

,query_id,item_id,ratings
0,Anton,All along the watchtower,1
1,Lesha,Tam gde klen shumit,1
2,Anna,Nothing else matters,1
3,Dasha,Nothing else matters,1
4,Sasha,Ty nesi menya reka,1


### Создание рекомендаций 

##### Порекомендуем всем две самые прослушиваемые песни

In [6]:
train.groupby('item_id').agg('count')['rating'].sort_values(ascending = False)

item_id
Nothing else matters        4
All along the watchtower    3
Kalinka-Malinka             2
Look at me now              2
Smells Like Teen Spirit     2
Tam gde klen shumit         2
Techet Reka Volga           2
Ty nesi menya reka          1
Name: rating, dtype: int64

In [7]:
pred = {'query_id': ['Anton', 'Anton', 'Lesha', 'Lesha', 'Anna', 'Anna', 'Dasha', 'Dasha', 'Sasha', 'Sasha'],
 'item_id': ['Nothing else matters', 'All along the watchtower', 'Nothing else matters', 'All along the watchtower',
             'Nothing else matters', 'All along the watchtower', 'Nothing else matters', 'All along the watchtower',
             'Nothing else matters', 'All along the watchtower'],
  'rating' : [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
pred = pd.DataFrame(pred)
pred

,query_id,item_id,rating
0,Anton,Nothing else matters,1
1,Anton,All along the watchtower,1
2,Lesha,Nothing else matters,1
3,Lesha,All along the watchtower,1
4,Anna,Nothing else matters,1
5,Anna,All along the watchtower,1
6,Dasha,Nothing else matters,1
7,Dasha,All along the watchtower,1
8,Sasha,Nothing else matters,1
9,Sasha,All along the watchtower,1


In [8]:
test

,query_id,item_id,ratings
0,Anton,All along the watchtower,1
1,Lesha,Tam gde klen shumit,1
2,Anna,Nothing else matters,1
3,Dasha,Nothing else matters,1
4,Sasha,Ty nesi menya reka,1


In [9]:
Anton_test = test.head(1)
Anton_pred = pred.head(2)

### Hit Rate

$
HitRate@k(u) =\mathbb{I}[|rel(u) \cap rec_k(u)| > 0]
$

$
HitRate@k =\frac 1N  \sum_{u=1}^N HitRate@k(u)$


In [10]:
def hit_rate(test:pd.DataFrame, predict:pd.DataFrame, topk:int) -> float:
    hit_rate_list = []
    user_pred = set(predict['query_id'].unique())
    user_test = set(test['query_id'].unique())
    user_union = set.intersection(user_pred, user_test)
    for user in user_union:
        test_seq = set(test[test['query_id'] == user]['item_id'].values)
        pred_seq = predict[predict['query_id'] == user]['item_id'].values
        if len(set(pred_seq[:topk]).intersection(test_seq)) > 0:
            hit_rate_list.append(1)
        else:
            hit_rate_list.append(0)
            
    no_test_users = user_test - user_union
    hit_rate_list.extend([0] * len(no_test_users))

    return np.mean(hit_rate_list)

In [11]:
Anton_test

,query_id,item_id,ratings
0,Anton,All along the watchtower,1


In [12]:
Anton_pred

,query_id,item_id,rating
0,Anton,Nothing else matters,1
1,Anton,All along the watchtower,1


$HitRate@1(u)$  для Антона  равен 0, так как первый элемент в рекомендациях не содержится в тесте.

In [13]:
print('Hit Rate = {}'.format(hit_rate(Anton_test, Anton_pred, topk=1)))

Hit Rate = 0.0


$HitRate@2(u)$ для Антона равен 1, так как только один из двух айтемов из рекомендаций (All along...) есть в тесте.

In [14]:
print('Hit Rate = {}'.format(hit_rate(Anton_test, Anton_pred, topk=2)))

Hit Rate = 1.0


Теперь посчитаем $HitRate@2$, т.е. для всех пользователей, усреднив  $HitRate@2(u)$ каждого.

In [15]:
print('Hit Rate = {}'.format(hit_rate(test, pred, topk=2)))

Hit Rate = 0.6


 Реализация из Replay

In [16]:
from replay.metrics import HitRate
print(HitRate(topk=2)(pred, test))

{'HitRate@2': 0.6}


### Coverage

$$Coverage@K=\frac{\left|\bigcup\limits_{u\in U} y_u\right|}{|I|}$$

In [17]:
def coverage(train:pd.DataFrame, predict:pd.DataFrame, topk:int) -> float:
    unique_item = set(train['item_id'])
    predict_topk = predict.groupby('query_id', group_keys=True)['item_id'].apply(lambda x: x.head(topk)).reset_index()['item_id']
    unique_predict = set(predict_topk)
    return len(unique_predict & unique_item) / len(unique_item)

In [18]:
train['item_id'].nunique()

8

In [19]:
Anton_pred

,query_id,item_id,rating
0,Anton,Nothing else matters,1
1,Anton,All along the watchtower,1


$Coverage@2(u)$ = 2 / 8, так как пересечение айтемов из трейна с айтемами из рекомендацияй равно 1, а всего уникальных айтемов 8.

In [20]:
print('Coverage = {}'.format(coverage(train, Anton_pred, topk=2))) 

Coverage = 0.25


Посчитаем $Coverage@2$, усреднив $Coverage@2(u)$ для каждого пользователя

In [21]:
print('Coverage = {}'.format(coverage(train, pred, topk=2))) 

Coverage = 0.25


Реализация из Replay

In [22]:
from replay.metrics import Coverage
print(Coverage(topk=2)(pred, train))

{'Coverage@2': 0.25}


### Precision

$Precision@k(u) = \frac{|rel(u) \cap rec_k(u)|}{k}$


$Precision@k =\frac 1N  \sum _{u=1}^N Precision@k(u)$


In [23]:
def precision(test:pd.DataFrame, predict:pd.DataFrame, topk:int) -> float:
    res = []
    user_pred = set(predict['query_id'].unique())
    user_test = set(test['query_id'].unique())
    user_union = set.intersection(user_pred, user_test)
    for user in user_union:
        test_seq = test[test['query_id'] == user]['item_id'].values
        pred_seq = predict[predict['query_id'] == user]['item_id'].values
        
        ans = len(set(pred_seq[:topk]) & set(test_seq))
        res.append(ans / topk)
            
    no_test_users = user_test - user_union
    res.extend([0] * len(no_test_users))

    return np.mean(res)

In [24]:
Anton_test

,query_id,item_id,ratings
0,Anton,All along the watchtower,1


In [25]:
Anton_pred

,query_id,item_id,rating
0,Anton,Nothing else matters,1
1,Anton,All along the watchtower,1


$Precision@1(u)$ для Антона равен 0, так как в рекомендациях нет айтема из теста.

In [26]:
print('Precision = {}'.format(precision(Anton_test, Anton_pred, topk=1)))

Precision = 0.0


$Precision@2(u)$ для Антона равен 0.5, так как ($test(u) \cap pred(u)$)=1, $topk$=2 
=> 

$(test(u) \cap pred(u) / topk$ = 0.5

In [27]:
print('Precision = {}'.format(precision(Anton_test, Anton_pred, topk=2)))

Precision = 0.5


Посчитаем $Precision@2$, усреднив $Precision@2(u)$ для каждого пользователя

In [28]:
print('Precision = {}'.format(precision(test, pred, topk=2)))

Precision = 0.3


 Реализация из Replay

In [29]:
from replay.metrics import Precision
print(Precision(topk=2)(pred, test))

{'Precision@2': 0.3}


### Recall

$Recall@k(u) = \frac{|rel(u) \cap rec_k(u)|}{|rel(u)|}$


$Recall@k =\frac 1N  \sum_{u=1}^N Recall@k(u)$


In [30]:
def recall(test: pd.DataFrame, predict : pd.DataFrame, topk:int) -> float:
    res = []
    user_pred = set(predict['query_id'].unique())
    user_test = set(test['query_id'].unique())
    user_union = set.intersection(user_pred, user_test)
    for user in user_union:
        test_seq = test[test['query_id'] == user]['item_id'].values
        pred_seq = predict[predict['query_id'] == user]['item_id'].values
            
        ans = len(set(pred_seq[:topk]).intersection(set(test_seq)))
        res.append(ans / len(set(test_seq)))
            
    no_test_users = user_test - user_union
    res.extend([0] * len(no_test_users))

    return np.mean(res)

$Recall@1(u)$ для Антона равен 0, так как в рекомендациях нет айтема из теста.

In [31]:
print('Recall = {}'.format(recall(Anton_test, Anton_pred, topk=1)))

Recall = 0.0


$Recall@2(u)$ для Антона равен 1, так как ($test(u) \cap pred(u)$)=1, $test(u)$=1 =>

$(test(u) \cap pred(u) / test(u)$ = 1

In [32]:
print('Recall = {}'.format(recall(Anton_test, Anton_pred, topk=2)))

Recall = 1.0


Посчитаем $Recall@2$, усреднив $Recall@2(u)$ для каждого пользователя

In [33]:
print('Recall = {}'.format(recall(test, pred, topk=2)))

Recall = 0.6


 Реализация из Replay

In [34]:
from replay.metrics import Recall
print(Recall(topk=2)(pred, test))

{'Recall@2': 0.6}


### Классы OfflineMetrics и experiments

Для подсчета множества метрик удобнее всего использовать OfflineMetrics. Он позволяется считать различные их модификации.

In [35]:
from replay.metrics import Median, ConfidenceInterval, PerUser, MAP, MRR, NDCG, RocAuc, Novelty, Surprisal, OfflineMetrics
metrics = [
     Precision(topk=2),
     Precision(topk=2, mode=PerUser()),
     Precision(topk=2, mode=Median()),
     Precision(topk=2, mode=ConfidenceInterval(alpha=0.95)),
     Recall(topk=2),
     MAP(topk=2),
     MRR(topk=2),
     NDCG(topk=2),
     HitRate(topk=2),
     RocAuc(topk=2),
     Coverage(topk=2),
     Novelty(topk=2),
     Surprisal(topk=2),
 ]

OfflineMetrics(metrics)(pred, test, train)

{'Precision@2': 0.3,
 'Precision-PerUser@2': {'Anna': 0.5,
  'Anton': 0.5,
  'Dasha': 0.5,
  'Lesha': 0.0,
  'Sasha': 0.0},
 'Precision-Median@2': 0.5,
 'Precision-ConfidenceInterval@2': 0.2400455838177655,
 'Recall@2': 0.6,
 'MAP@2': 0.5,
 'MRR@2': 0.5,
 'NDCG@2': 0.5261859507142915,
 'HitRate@2': 0.6,
 'RocAuc@2': 0.4,
 'Coverage@2': 0.25,
 'Novelty@2': 0.3,
 'Surprisal@2': 0.22802034468361435}

Для сравнения метрик двух моделей удобно использовать модуль Experiment. Давайте сравним модель, которая рекомендует два самых популярных айтема с той, которая рекомендует всем два случайных.

In [36]:
random_rec = pd.DataFrame({'query_id': ['Anton', 'Anton', 'Lesha', 'Lesha', 'Anna', 'Anna', 'Dasha', 'Dasha', 'Sasha', 'Sasha']})
random_rec['item_id'] = pd.Series(test['item_id'].unique()).sample(n=10, replace=True).values
random_rec['rating'] = np.ones(10)


In [37]:
from replay.metrics import Experiment
ex = Experiment([NDCG([2]), Precision(2), Coverage(2)], test, train)
ex.add_result("randomrec@2", random_rec)
ex.add_result("poprec@2", pred)


/home/apembek/anaconda3/envs/py39/lib/python3.9/site-packages/replay/metrics/base_metric.py:80: MetricDuplicatesWarning: The recommendations contain duplicated users and items.The metrics may be higher than the actual ones.
  warnings.warn(
/home/apembek/anaconda3/envs/py39/lib/python3.9/site-packages/replay/metrics/base_metric.py:80: MetricDuplicatesWarning: The recommendations contain duplicated users and items.The metrics may be higher than the actual ones.
  warnings.warn(
/home/apembek/anaconda3/envs/py39/lib/python3.9/site-packages/replay/metrics/base_metric.py:80: MetricDuplicatesWarning: The recommendations contain duplicated users and items.The metrics may be higher than the actual ones.
  warnings.warn(


In [38]:
ex.results

,NDCG@2,Precision@2,Coverage@2
randomrec@2,0.252372,0.2,0.50
poprec@2,0.526186,0.3,0.25


Посчитаем наши метрики на большом датасете

In [39]:
from rs_datasets import MovieLens
ml = MovieLens('1m')
data = ml.ratings.rename(columns={'user_id':'query_id'})
data.head()

,query_id,item_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [40]:
from replay.splitters.random_splitter import RandomSplitter
train, test = RandomSplitter(test_size=0.1).split(data)

In [41]:
random_item = pd.Series(train['item_id'].unique()).sample(n=test.query_id.nunique()*10,replace=True, random_state=17)

In [42]:
predict = pd.DataFrame()
predict['query_id'] = np.repeat(test['query_id'].unique(), 10)
predict['item_id'] = list(random_item)
predict['rating'] = [10 - x % 10 for x in range(len(predict))]
predict

,query_id,item_id,rating
0,1,2586,10
1,1,212,9
2,1,355,8
3,1,2182,7
4,1,2891,6
...,...,...,...
59475,6040,2118,5
59476,6040,746,4
59477,6040,2998,3
59478,6040,2644,2


In [43]:
assert coverage(train, predict, topk=1) == Coverage(topk=1)(predict, train)['Coverage@1']
assert hit_rate(test, predict, topk=1) == HitRate(topk=1)(predict, test)['HitRate@1']
assert precision(test, predict, topk=1) == Precision(topk=1)(predict, test)['Precision@1']
assert recall(test, predict, topk=1) == Recall(topk=1)(predict, test)['Recall@1']

/home/apembek/anaconda3/envs/py39/lib/python3.9/site-packages/replay/metrics/base_metric.py:80: MetricDuplicatesWarning: The recommendations contain duplicated users and items.The metrics may be higher than the actual ones.
  warnings.warn(
/home/apembek/anaconda3/envs/py39/lib/python3.9/site-packages/replay/metrics/base_metric.py:80: MetricDuplicatesWarning: The recommendations contain duplicated users and items.The metrics may be higher than the actual ones.
  warnings.warn(
/home/apembek/anaconda3/envs/py39/lib/python3.9/site-packages/replay/metrics/base_metric.py:80: MetricDuplicatesWarning: The recommendations contain duplicated users and items.The metrics may be higher than the actual ones.
  warnings.warn(
/home/apembek/anaconda3/envs/py39/lib/python3.9/site-packages/replay/metrics/base_metric.py:80: MetricDuplicatesWarning: The recommendations contain duplicated users and items.The metrics may be higher than the actual ones.
  warnings.warn(


In [44]:
assert coverage(train, predict, topk=10) == Coverage(topk=10)(predict, train)['Coverage@10']
assert hit_rate(test, predict, topk=10) == HitRate(topk=10)(predict, test)['HitRate@10']
assert precision(test, predict, topk=10) == Precision(topk=10)(predict, test)['Precision@10']
assert recall(test, predict, topk=10) == Recall(topk=10)(predict, test)['Recall@10']

/home/apembek/anaconda3/envs/py39/lib/python3.9/site-packages/replay/metrics/base_metric.py:80: MetricDuplicatesWarning: The recommendations contain duplicated users and items.The metrics may be higher than the actual ones.
  warnings.warn(
/home/apembek/anaconda3/envs/py39/lib/python3.9/site-packages/replay/metrics/base_metric.py:80: MetricDuplicatesWarning: The recommendations contain duplicated users and items.The metrics may be higher than the actual ones.
  warnings.warn(
/home/apembek/anaconda3/envs/py39/lib/python3.9/site-packages/replay/metrics/base_metric.py:80: MetricDuplicatesWarning: The recommendations contain duplicated users and items.The metrics may be higher than the actual ones.
  warnings.warn(
/home/apembek/anaconda3/envs/py39/lib/python3.9/site-packages/replay/metrics/base_metric.py:80: MetricDuplicatesWarning: The recommendations contain duplicated users and items.The metrics may be higher than the actual ones.
  warnings.warn(


Сравним скорость работы наших метрик и из Replay

In [45]:
import time

In [46]:
start_time = time.time()
coverage(train, predict, topk=1)
hit_rate(test, predict, topk=1)
precision(test, predict, topk=1)
recall(test, predict, topk=1) 
print(f"Затрачено времени: {time.time() - start_time} сек")

Затрачено времени: 13.178834915161133 сек


In [47]:
start_time = time.time()
Coverage(topk=1)(predict, train)['Coverage@1']
HitRate(topk=1)(predict, test)['HitRate@1']
Precision(topk=1)(predict, test)['Precision@1']
Recall(topk=1)(predict, test)['Recall@1']
print(f"Затрачено времени: {time.time() - start_time} сек")

/home/apembek/anaconda3/envs/py39/lib/python3.9/site-packages/replay/metrics/base_metric.py:80: MetricDuplicatesWarning: The recommendations contain duplicated users and items.The metrics may be higher than the actual ones.
  warnings.warn(
/home/apembek/anaconda3/envs/py39/lib/python3.9/site-packages/replay/metrics/base_metric.py:80: MetricDuplicatesWarning: The recommendations contain duplicated users and items.The metrics may be higher than the actual ones.
  warnings.warn(
/home/apembek/anaconda3/envs/py39/lib/python3.9/site-packages/replay/metrics/base_metric.py:80: MetricDuplicatesWarning: The recommendations contain duplicated users and items.The metrics may be higher than the actual ones.
  warnings.warn(


Затрачено времени: 1.1390430927276611 сек


/home/apembek/anaconda3/envs/py39/lib/python3.9/site-packages/replay/metrics/base_metric.py:80: MetricDuplicatesWarning: The recommendations contain duplicated users and items.The metrics may be higher than the actual ones.
  warnings.warn(
